### Definindo uma função para montar um ADLS com um ponto de montagem com ADLS SAS 

In [0]:
storageAccountName = "datalake922b9abd80170b5b"
storageAccountAccessKey = ""
sasToken= "sv=2024-11-04&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2025-06-30T08:33:56Z&st=2025-06-18T00:33:56Z&spr=https&sig=MJQBJIyhWinObTmsfSK5xjUc7F5GVJQ9dQnz8o7TfF4%3D"

def mount_adls(blobContainerName):
    try:
      dbutils.fs.mount(
        source = "wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
        mount_point = f"/mnt/{storageAccountName}/{blobContainerName}",
        #extra_configs = {'fs.azure.account.key.' + storageAccountName + '.blob.core.windows.net': storageAccountAccessKey}
        extra_configs = {'fs.azure.sas.' + blobContainerName + '.' + storageAccountName + '.blob.core.windows.net': sasToken}
      )
      print("OK!")
    except Exception as e:
      print("Falha", e)

###Montando todos os containers

In [0]:
#dbutils.fs.unmount('/mnt/datalake922b9abd80170b5b/landing-zone')
#dbutils.fs.unmount('/mnt/datalake922b9abd80170b5b/bronze')
#dbutils.fs.unmount('/mnt/datalake922b9abd80170b5b/silver')
#dbutils.fs.unmount('/mnt/datalake922b9abd80170b5b/gold')

/mnt/datalake922b9abd80170b5b/landing-zone has been unmounted.
/mnt/datalake922b9abd80170b5b/bronze has been unmounted.
/mnt/datalake922b9abd80170b5b/silver has been unmounted.
/mnt/datalake922b9abd80170b5b/gold has been unmounted.
Out[7]: True

In [0]:
#mount_adls('lading-zone')
#mount_adls('bronze')
#mount_adls('silver')
#mount_adls('gold')

Falha An error occurred while calling o1858.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/datalake922b9abd80170b5b/lading-zone; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/datalake922b9abd80170b5b/lading-zone
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:128)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:68)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.createOrUpdateMount(DBUtilsCore.scala:756)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:776)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:750)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/datalake922b9abd80170b5b/lading-zone
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$insertMount$1(MetadataManager.scala:860)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$modifyAndVerify$2(MetadataManager.scala:1242)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.withRetries(MetadataManager.scala:1015)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.modifyAndVerify(MetadataManager.scala:1231)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:868)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:131)
	at com.databricks.backend.daemon.data.server.handler.DbfsRequestHandler.receive(DbfsRequestHandler.scala:16)
	at com.databricks.backend.daemon.data.server.handler.DbfsRequestHandler.receive$(DbfsRequestHandler.scala:15)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:39)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:54)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:53)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:53)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend.$anonfun$handleOtherRpc$2(DbfsServerBackend.scala:599)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:49)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:290)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:286)
	at com.databricks.logging.AttributionContextTracing.withAttributionContext(AttributionContextTracing.scala:47)
	at com.databricks.logging.AttributionContextTracing.withAttributionContext$(AttributionContextTracing.scala:44)
	at com.databricks.rpc.ServerBackend.withAttributionContext(ServerBackend.scala:22)
	at com.databricks.logging.AttributionContextTracing.withAttributionTags(AttributionContextTracing.scala:96)
	at com.databrick

###Mostrando os pontos de montagem no cluster Databricks

###Gerando um dataframe dos delta lake no container gold do Azure Data Lake Storage

In [0]:
df_avaliacoes          = spark.read.format('delta').load(f"/mnt/{storageAccountName}/silver/ecommerce/avaliacoes")
df_categorias          = spark.read.format('delta').load(f"/mnt/{storageAccountName}/silver/ecommerce/categorias")
df_clientes            = spark.read.format('delta').load(f"/mnt/{storageAccountName}/silver/ecommerce/clientes")
df_enderecos_cliente  = spark.read.format('delta').load(f"/mnt/{storageAccountName}/silver/ecommerce/enderecos_cliente")
df_entregas            = spark.read.format('delta').load(f"/mnt/{storageAccountName}/silver/ecommerce/entregas")
df_estoque             = spark.read.format('delta').load(f"/mnt/{storageAccountName}/silver/ecommerce/estoque")
df_formas_pagamento    = spark.read.format('delta').load(f"/mnt/{storageAccountName}/silver/ecommerce/formas_pagamento")
df_itens_pedidos       = spark.read.format('delta').load(f"/mnt/{storageAccountName}/silver/ecommerce/itens_pedido")
df_pagamentos          = spark.read.format('delta').load(f"/mnt/{storageAccountName}/silver/ecommerce/pagamentos")
df_pedidos             = spark.read.format('delta').load(f"/mnt/{storageAccountName}/silver/ecommerce/pedidos")
df_produtos            = spark.read.format('delta').load(f"/mnt/{storageAccountName}/silver/ecommerce/produtos")
df_transportadoras     = spark.read.format('delta').load(f"/mnt/{storageAccountName}/silver/ecommerce/transportadoras")
df_vendedores          = spark.read.format('delta').load(f"/mnt/{storageAccountName}/silver/ecommerce/vendedores")

### Adicionando metadados de data e hora de processamento e nome do arquivo de origem

In [0]:
from pyspark.sql.functions import current_timestamp, lit


df_avaliacoes          = df_avaliacoes.withColumn("data_hora_gold", current_timestamp()).withColumn("nome_arquivo", lit("avaliacoes"))
df_categorias          = df_categorias.withColumn("data_hora_gold", current_timestamp()).withColumn("nome_arquivo", lit("categorias"))
df_clientes            = df_clientes.withColumn("data_hora_gold", current_timestamp()).withColumn("nome_arquivo", lit("clientes"))
df_entregas            = df_entregas.withColumn("data_hora_gold", current_timestamp()).withColumn("nome_arquivo", lit("entregas"))
df_estoque             = df_estoque.withColumn("data_hora_gold", current_timestamp()).withColumn("nome_arquivo", lit("estoque"))
df_formas_pagamento    = df_formas_pagamento.withColumn("data_hora_gold", current_timestamp()).withColumn("nome_arquivo", lit("formas_pagamento"))
df_itens_pedidos       = df_itens_pedidos.withColumn("data_hora_gold", current_timestamp()).withColumn("nome_arquivo", lit("itens_pedidos"))
df_pagamentos          = df_pagamentos.withColumn("data_hora_gold", current_timestamp()).withColumn("nome_arquivo", lit("pagamentos"))
df_pedidos             = df_pedidos.withColumn("data_hora_gold", current_timestamp()).withColumn("nome_arquivo", lit("pedidos"))
df_produtos            = df_produtos.withColumn("data_hora_gold", current_timestamp()).withColumn("nome_arquivo", lit("produtos"))
df_transportadoras     = df_transportadoras.withColumn("data_hora_gold", current_timestamp()).withColumn("nome_arquivo", lit("transportadoras"))
df_vendedores          = df_vendedores.withColumn("data_hora_gold", current_timestamp()).withColumn("nome_arquivo", lit("vendedores"))

In [0]:
from pyspark.sql.functions import current_timestamp, lit

df_enderecos_cliente = (
    df_enderecos_cliente
    .withColumnRenamed("CCODIGOADE", "CIDADE")  # renomeia a coluna errada para a correta
    .withColumn("data_hora_gold", current_timestamp())
    .withColumn("nome_arquivo", lit("enderecos_cliente"))
)

df_itens_pedidos = (
    df_itens_pedidos
    .withColumnRenamed("QUANTCODIGOADE", "QUANTIDADE")  # renomeia a coluna errada para a correta
    .withColumn("data_hora_gold", current_timestamp())
    .withColumn("nome_arquivo", lit("enderecos_cliente"))
)




In [0]:
%sql
drop table if exists dim_clientes

In [0]:
%sql
CREATE TABLE IF NOT EXISTS dim_clientes (
  ID BIGINT GENERATED ALWAYS AS IDENTITY,
  CODIGO_CLIENTE INT,
  NOME STRING,
  EMAIL STRING,
  TELEFONE STRING,
  CIDADE STRING,
  ESTADO STRING,
  DATA_CADASTRO TIMESTAMP
)
USING delta
LOCATION '/mnt/datalake922b9abd80170b5b/gold/ecommerce/dim_clientes';


In [0]:
df_clientes.createOrReplaceTempView("silver_clientes")
df_enderecos_cliente.createOrReplaceTempView("silver_enderecos_cliente")


In [0]:
%sql
-- Criar TEMP VIEW com os dados de origem
CREATE OR REPLACE TEMP VIEW dim_clientes_temp AS
SELECT
    c.CODIGO AS CODIGO_CLIENTE,
    c.NOME,
    c.EMAIL,
    c.TELEFONE,
    ec.CIDADE,
    ec.ESTADO,
    c.DATA_CADASTRO
FROM silver_clientes c
LEFT JOIN silver_enderecos_cliente ec ON c.CODIGO = ec.CODIGO_CLIENTE;

-- MERGE na tabela de dimensão usando a chave natural
MERGE INTO dim_clientes AS target
USING dim_clientes_temp AS source
ON target.CODIGO_CLIENTE = source.CODIGO_CLIENTE
WHEN MATCHED THEN
  UPDATE SET
    target.NOME = source.NOME,
    target.EMAIL = source.EMAIL,
    target.TELEFONE = source.TELEFONE,
    target.CIDADE = source.CIDADE,
    target.ESTADO = source.ESTADO,
    target.DATA_CADASTRO = source.DATA_CADASTRO
WHEN NOT MATCHED THEN
  INSERT (CODIGO_CLIENTE, NOME, EMAIL, TELEFONE, CIDADE, ESTADO, DATA_CADASTRO)
  VALUES (source.CODIGO_CLIENTE, source.NOME, source.EMAIL, source.TELEFONE, source.CIDADE, source.ESTADO, source.DATA_CADASTRO);


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
13710,0,0,13710


In [0]:
%sql
drop table if exists dim_produtos

In [0]:
%sql
CREATE TABLE IF NOT EXISTS dim_produtos (
  ID BIGINT GENERATED ALWAYS AS IDENTITY,  -- Chave substituta (SK)
  CODIGO_PRODUTO INT,                      -- Chave natural do sistema transacional
  NOME STRING,
  DESCRICAO STRING,
  PRECO DECIMAL(10,2),
  CATEGORIA STRING,
  DATA_CADASTRO TIMESTAMP
)
USING delta
LOCATION '/mnt/datalake922b9abd80170b5b/gold/ecommerce/dim_produtos';


In [0]:
df_produtos.createOrReplaceTempView("silver_produtos")
df_categorias.createOrReplaceTempView("silver_categorias")


In [0]:
%sql
-- Criar ou substituir a TEMP VIEW com os dados transformados
CREATE OR REPLACE TEMP VIEW dim_produtos_temp AS
SELECT
    p.CODIGO AS CODIGO_PRODUTO,
    p.NOME,
    p.DESCRICAO,
    p.PRECO,
    c.NOME AS CATEGORIA,
    p.DATA_CADASTRO
FROM silver_produtos p
LEFT JOIN silver_categorias c ON p.CODIGO_CATEGORIA = c.CODIGO;

-- Realizar o MERGE na tabela de dimensão
MERGE INTO dim_produtos AS target
USING dim_produtos_temp AS source
ON target.CODIGO_PRODUTO = source.CODIGO_PRODUTO
WHEN MATCHED THEN
  UPDATE SET
    target.NOME = source.NOME,
    target.DESCRICAO = source.DESCRICAO,
    target.PRECO = source.PRECO,
    target.CATEGORIA = source.CATEGORIA,
    target.DATA_CADASTRO = source.DATA_CADASTRO
WHEN NOT MATCHED THEN
  INSERT (CODIGO_PRODUTO, NOME, DESCRICAO, PRECO, CATEGORIA, DATA_CADASTRO)
  VALUES (source.CODIGO_PRODUTO, source.NOME, source.DESCRICAO, source.PRECO, source.CATEGORIA, source.DATA_CADASTRO);


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
30001,0,0,30001


In [0]:
%sql
drop table if exists dim_vendedores

In [0]:
%sql
CREATE TABLE IF NOT EXISTS dim_vendedores (
  ID BIGINT GENERATED ALWAYS AS IDENTITY,   -- Chave substituta (SK)
  CODIGO_VENDEDOR INT,                      -- Chave natural do sistema de origem
  NOME STRING,
  EMAIL STRING,
  TELEFONE STRING,
  DATA_CADASTRO TIMESTAMP
)
USING delta
LOCATION '/mnt/datalake922b9abd80170b5b/gold/ecommerce/dim_vendedores';


In [0]:
df_vendedores.createOrReplaceTempView("silver_vendedores")


In [0]:
%sql
-- Criar ou substituir a TEMP VIEW com os dados transformados
CREATE OR REPLACE TEMP VIEW dim_vendedores_temp AS
SELECT
    CODIGO AS CODIGO_VENDEDOR,
    NOME,
    EMAIL,
    TELEFONE,
    DATA_CADASTRO
FROM silver_vendedores;

-- Realizar o MERGE na tabela de dimensão
MERGE INTO dim_vendedores AS target
USING dim_vendedores_temp AS source
ON target.CODIGO_VENDEDOR = source.CODIGO_VENDEDOR
WHEN MATCHED THEN
  UPDATE SET
    target.NOME = source.NOME,
    target.EMAIL = source.EMAIL,
    target.TELEFONE = source.TELEFONE,
    target.DATA_CADASTRO = source.DATA_CADASTRO
WHEN NOT MATCHED THEN
  INSERT (CODIGO_VENDEDOR, NOME, EMAIL, TELEFONE, DATA_CADASTRO)
  VALUES (source.CODIGO_VENDEDOR, source.NOME, source.EMAIL, source.TELEFONE, source.DATA_CADASTRO);


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
1001,0,0,1001


In [0]:
%sql
drop table if exists dim_formas_pagamento

In [0]:
%sql
CREATE TABLE IF NOT EXISTS dim_formas_pagamento (
  ID BIGINT GENERATED ALWAYS AS IDENTITY,  -- Chave substituta (SK)
  CODIGO_FORMA INT,                        -- Chave natural (caso exista)
  DESCRICAO STRING
)
USING delta
LOCATION '/mnt/datalake922b9abd80170b5b/gold/ecommerce/dim_formas_pagamento';


In [0]:
df_formas_pagamento.createOrReplaceTempView("silver_formas_pagamento")


In [0]:
%sql
-- Criar ou substituir a TEMP VIEW com dados da origem
CREATE OR REPLACE TEMP VIEW dim_formas_pagamento_temp AS
SELECT
    CODIGO AS CODIGO_FORMA,
    DESCRICAO
FROM silver_formas_pagamento;

-- MERGE na dimensão dim_formas_pagamento
MERGE INTO dim_formas_pagamento AS target
USING dim_formas_pagamento_temp AS source
ON target.CODIGO_FORMA = source.CODIGO_FORMA
WHEN MATCHED THEN
  UPDATE SET
    target.DESCRICAO = source.DESCRICAO
WHEN NOT MATCHED THEN
  INSERT (CODIGO_FORMA, DESCRICAO)
  VALUES (source.CODIGO_FORMA, source.DESCRICAO);


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
11,0,0,11


In [0]:
%sql
drop table if exists dim_tempo

In [0]:
from pyspark.sql.functions import expr, date_format, year, month, dayofmonth, dayofweek, when, col

data_inicial = "2022-01-01"
data_final = "2025-12-31"

num_dias = spark.sql(f"SELECT datediff('{data_final}', '{data_inicial}')").collect()[0][0]

df_calendario = spark.range(0, num_dias + 1) \
    .selectExpr(f"date_add(to_date('{data_inicial}'), CAST(id AS INT)) AS Data")

# Criar colunas nome mês e dia da semana com when (case when)
df_tempo = df_calendario.select(
    col("Data"),
    year("Data").alias("Ano"),
    month("Data").alias("Mes"),
    when(month("Data") == 1, "JANEIRO")
    .when(month("Data") == 2, "FEVEREIRO")
    .when(month("Data") == 3, "MARCO")
    .when(month("Data") == 4, "ABRIL")
    .when(month("Data") == 5, "MAIO")
    .when(month("Data") == 6, "JUNHO")
    .when(month("Data") == 7, "JULHO")
    .when(month("Data") == 8, "AGOSTO")
    .when(month("Data") == 9, "SETEMBRO")
    .when(month("Data") == 10, "OUTUBRO")
    .when(month("Data") == 11, "NOVEMBRO")
    .when(month("Data") == 12, "DEZEMBRO")
    .alias("NomeMes"),
    dayofmonth("Data").alias("Dia"),
    when(dayofweek("Data") == 1, "DOMINGO")
    .when(dayofweek("Data") == 2, "SEGUNDA-FEIRA")
    .when(dayofweek("Data") == 3, "TERCA-FEIRA")
    .when(dayofweek("Data") == 4, "QUARTA-FEIRA")
    .when(dayofweek("Data") == 5, "QUINTA-FEIRA")
    .when(dayofweek("Data") == 6, "SEXTA-FEIRA")
    .when(dayofweek("Data") == 7, "SABADO")
    .alias("NomeDiaSemana"),
    dayofweek("Data").alias("NumeroDiaSemana"),
    date_format("Data", "yyyyMMdd").cast("int").alias("ID")
)

df_tempo.display()

df_tempo.write.mode("overwrite")\
    .option("path", f"/mnt/{storageAccountName}/gold/ecommerce/dim_tempo")\
    .saveAsTable("dim_tempo")


Data,Ano,Mes,NomeMes,Dia,NomeDiaSemana,NumeroDiaSemana,ID
2022-01-01,2022,1,JANEIRO,1,SABADO,7,20220101
2022-01-02,2022,1,JANEIRO,2,DOMINGO,1,20220102
2022-01-03,2022,1,JANEIRO,3,SEGUNDA-FEIRA,2,20220103
2022-01-04,2022,1,JANEIRO,4,TERCA-FEIRA,3,20220104
2022-01-05,2022,1,JANEIRO,5,QUARTA-FEIRA,4,20220105
2022-01-06,2022,1,JANEIRO,6,QUINTA-FEIRA,5,20220106
2022-01-07,2022,1,JANEIRO,7,SEXTA-FEIRA,6,20220107
2022-01-08,2022,1,JANEIRO,8,SABADO,7,20220108
2022-01-09,2022,1,JANEIRO,9,DOMINGO,1,20220109
2022-01-10,2022,1,JANEIRO,10,SEGUNDA-FEIRA,2,20220110


In [0]:
%sql
drop table if exists dim_entregas

In [0]:
%sql
CREATE TABLE IF NOT EXISTS dim_entregas (
  ID BIGINT GENERATED ALWAYS AS IDENTITY,  -- Chave substituta (SK)
  CODIGO_ENTREGA INT,                       -- Chave natural (caso exista)
  TRANSPORTADORA STRING,
  STATUS STRING,
  DATA_ENVIO TIMESTAMP,
  DATA_ENTREGA TIMESTAMP
)
USING delta
LOCATION '/mnt/datalake922b9abd80170b5b/gold/ecommerce/dim_entregas';


In [0]:
df_entregas.createOrReplaceTempView("silver_entregas")
df_transportadoras.createOrReplaceTempView("silver_transportadoras")


In [0]:
%sql
-- Criar ou substituir a TEMP VIEW com os dados de origem
CREATE OR REPLACE TEMP VIEW dim_entregas_temp AS
SELECT
    e.CODIGO AS CODIGO_ENTREGA,
    t.NOME AS TRANSPORTADORA,
    e.STATUS,
    e.DATA_ENVIO,
    e.DATA_ENTREGA
FROM silver_entregas e
LEFT JOIN silver_transportadoras t ON e.CODIGO_TRANSPORTADORA = t.CODIGO;

-- Executar o MERGE na tabela de dimensão
MERGE INTO dim_entregas AS target
USING dim_entregas_temp AS source
ON target.CODIGO_ENTREGA = source.CODIGO_ENTREGA
WHEN MATCHED THEN
  UPDATE SET
    target.TRANSPORTADORA = source.TRANSPORTADORA,
    target.STATUS = source.STATUS,
    target.DATA_ENVIO = source.DATA_ENVIO,
    target.DATA_ENTREGA = source.DATA_ENTREGA
WHEN NOT MATCHED THEN
  INSERT (CODIGO_ENTREGA, TRANSPORTADORA, STATUS, DATA_ENVIO, DATA_ENTREGA)
  VALUES (source.CODIGO_ENTREGA, source.TRANSPORTADORA, source.STATUS, source.DATA_ENVIO, source.DATA_ENTREGA);


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
25001,0,0,25001


In [0]:
%sql
drop table if exists fato_vendas

In [0]:
%sql
CREATE TABLE IF NOT EXISTS fato_vendas (
  ID BIGINT GENERATED ALWAYS AS IDENTITY,  -- SK automática
  CLIENTE_SK INT,
  PRODUTO_SK INT,
  VENDEDOR_SK INT,
  TEMPO_SK INT,
  FORMA_PAGAMENTO_SK INT,
  ENTREGA_SK INT,
  QUANTIDADE INT,
  PRECO_UNITARIO DECIMAL(10,2),
  VALOR_TOTAL DECIMAL(12,2),
  TEMPO_ENTREGA_DIAS INT,
  NOTA_AVALIACAO INT
)
USING delta
LOCATION '/mnt/datalake922b9abd80170b5b/gold/ecommerce/fato_vendas';


In [0]:
df_pedidos.createOrReplaceTempView("pedidos")
df_itens_pedidos.createOrReplaceTempView("itens_pedidos")
df_pagamentos.createOrReplaceTempView("pagamentos")
df_avaliacoes.createOrReplaceTempView("avaliacoes")
df_produtos.createOrReplaceTempView("produtos")


In [0]:
spark.read.format("delta").load("/mnt/datalake922b9abd80170b5b/gold/ecommerce/dim_clientes").createOrReplaceTempView("dim_clientes")
spark.read.format("delta").load("/mnt/datalake922b9abd80170b5b/gold/ecommerce/dim_produtos").createOrReplaceTempView("dim_produtos")
spark.read.format("delta").load("/mnt/datalake922b9abd80170b5b/gold/ecommerce/dim_vendedores").createOrReplaceTempView("dim_vendedores")
spark.read.format("delta").load("/mnt/datalake922b9abd80170b5b/gold/ecommerce/dim_tempo").createOrReplaceTempView("dim_tempo")
spark.read.format("delta").load("/mnt/datalake922b9abd80170b5b/gold/ecommerce/dim_formas_pagamento").createOrReplaceTempView("dim_formas_pagamento")
spark.read.format("delta").load("/mnt/datalake922b9abd80170b5b/gold/ecommerce/dim_entregas").createOrReplaceTempView("dim_entregas")


In [0]:
%sql
MERGE INTO fato_vendas AS target
USING (
  SELECT
    dc.ID AS CLIENTE_SK,
    dp.ID AS PRODUTO_SK,
    dv.ID AS VENDEDOR_SK,
    dt.ID AS TEMPO_SK,
    dfp.ID AS FORMA_PAGAMENTO_SK,
    de.ID AS ENTREGA_SK,
    ip.QUANTIDADE,
    ip.PRECO_UNITARIO,
    ip.QUANTIDADE * ip.PRECO_UNITARIO AS VALOR_TOTAL,
    DATEDIFF(de.DATA_ENTREGA, de.DATA_ENVIO) AS TEMPO_ENTREGA_DIAS,
    av.NOTA AS NOTA_AVALIACAO
  FROM pedidos p
  INNER JOIN itens_pedidos ip ON ip.CODIGO_PEDIDO = p.CODIGO
  INNER JOIN dim_clientes dc ON dc.CODIGO_CLIENTE = p.CODIGO_CLIENTE
  INNER JOIN dim_produtos dp ON dp.CODIGO_PRODUTO = ip.CODIGO_PRODUTO
  INNER JOIN produtos pr ON pr.CODIGO = ip.CODIGO_PRODUTO
  INNER JOIN dim_vendedores dv ON dv.CODIGO_VENDEDOR = pr.CODIGO_VENDEDOR
  INNER JOIN dim_tempo dt ON dt.Data = CAST(p.DATA_PEDCODIGOO AS DATE)
  LEFT JOIN pagamentos pg ON pg.CODIGO_PEDIDO = p.CODIGO
  LEFT JOIN dim_formas_pagamento dfp ON dfp.CODIGO_FORMA = pg.CODIGO
  LEFT JOIN dim_entregas de ON de.CODIGO_ENTREGA = p.CODIGO_ENDERECO_ENTREGA
  LEFT JOIN (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY CODIGO_PRODUTO, CODIGO_CLIENTE ORDER BY DATA_AVALIACAO DESC) AS rn
    FROM avaliacoes
  ) av ON av.CODIGO_PRODUTO = ip.CODIGO_PRODUTO
       AND av.CODIGO_CLIENTE = p.CODIGO_CLIENTE
       AND av.rn = 1
) AS source
ON 
  target.CLIENTE_SK = source.CLIENTE_SK
  AND target.PRODUTO_SK = source.PRODUTO_SK
  AND target.VENDEDOR_SK = source.VENDEDOR_SK
  AND target.TEMPO_SK = source.TEMPO_SK
  AND target.FORMA_PAGAMENTO_SK = source.FORMA_PAGAMENTO_SK
  AND target.ENTREGA_SK = source.ENTREGA_SK
  AND target.QUANTIDADE = source.QUANTIDADE
  AND target.PRECO_UNITARIO = source.PRECO_UNITARIO
WHEN NOT MATCHED THEN
INSERT (
  CLIENTE_SK,
  PRODUTO_SK,
  VENDEDOR_SK,
  TEMPO_SK,
  FORMA_PAGAMENTO_SK,
  ENTREGA_SK,
  QUANTIDADE,
  PRECO_UNITARIO,
  VALOR_TOTAL,
  TEMPO_ENTREGA_DIAS,
  NOTA_AVALIACAO
)
VALUES (
  source.CLIENTE_SK,
  source.PRODUTO_SK,
  source.VENDEDOR_SK,
  source.TEMPO_SK,
  source.FORMA_PAGAMENTO_SK,
  source.ENTREGA_SK,
  source.QUANTIDADE,
  source.PRECO_UNITARIO,
  source.VALOR_TOTAL,
  source.TEMPO_ENTREGA_DIAS,
  source.NOTA_AVALIACAO
);


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
114115,0,0,114115


In [0]:
%sql
SELECT
  dc.ID AS CLIENTE_SK,
  dp.ID AS PRODUTO_SK,
  dv.ID AS VENDEDOR_SK,
  dt.ID AS TEMPO_SK,
  dfp.ID AS FORMA_PAGAMENTO_SK,
  de.ID AS ENTREGA_SK,
  ip.QUANTIDADE,
  ip.PRECO_UNITARIO,
  ip.QUANTIDADE * ip.PRECO_UNITARIO AS VALOR_TOTAL,
  DATEDIFF(de.DATA_ENTREGA, de.DATA_ENVIO) AS TEMPO_ENTREGA_DIAS,
  av.NOTA AS NOTA_AVALIACAO
FROM pedidos p
INNER JOIN itens_pedidos ip ON ip.CODIGO_PEDIDO = p.CODIGO
INNER JOIN dim_clientes dc ON dc.CODIGO_CLIENTE = p.CODIGO_CLIENTE
INNER JOIN dim_produtos dp ON dp.CODIGO_PRODUTO = ip.CODIGO_PRODUTO
INNER JOIN produtos pr ON pr.CODIGO = ip.CODIGO_PRODUTO
INNER JOIN dim_vendedores dv ON dv.CODIGO_VENDEDOR = pr.CODIGO_VENDEDOR
INNER JOIN dim_tempo dt ON dt.Data = CAST(p.DATA_PEDCODIGOO AS DATE)
INNER JOIN pagamentos pg ON pg.CODIGO_PEDIDO = p.CODIGO
INNER JOIN dim_formas_pagamento dfp ON dfp.CODIGO_FORMA = pg.CODIGO
LEFT JOIN dim_entregas de ON de.CODIGO_ENTREGA = p.CODIGO_ENDERECO_ENTREGA
LEFT JOIN (
  SELECT *,
    ROW_NUMBER() OVER (PARTITION BY CODIGO_PRODUTO, CODIGO_CLIENTE ORDER BY DATA_AVALIACAO DESC) AS rn
  FROM avaliacoes
) av ON av.CODIGO_PRODUTO = ip.CODIGO_PRODUTO
     AND av.CODIGO_CLIENTE = p.CODIGO_CLIENTE
     AND av.rn = 1


CLIENTE_SK,PRODUTO_SK,VENDEDOR_SK,TEMPO_SK,FORMA_PAGAMENTO_SK,ENTREGA_SK,QUANTIDADE,PRECO_UNITARIO,VALOR_TOTAL,TEMPO_ENTREGA_DIAS,NOTA_AVALIACAO
5108,698,834,20221011,10,6872,5,318.84,1594.1999999999998,0,null
5107,698,834,20221011,10,6872,5,318.84,1594.1999999999998,0,null
5106,698,834,20221011,10,6872,5,318.84,1594.1999999999998,0,null
5105,698,834,20221011,10,6872,5,318.84,1594.1999999999998,0,null
7419,29518,591,20240713,5,5999,2,701.85,1403.7,5,null
7418,29518,591,20240713,5,5999,2,701.85,1403.7,5,null
7417,29518,591,20240713,5,5999,2,701.85,1403.7,5,null
7416,29518,591,20240713,5,5999,2,701.85,1403.7,5,null
5080,7319,439,20221116,3,6468,2,453.76,907.52,5,null
7419,3157,139,20240713,5,5999,2,530.49,1060.98,5,null


In [0]:
%sql
SELECT COUNT(*) 
FROM pedidos p
LEFT JOIN pagamentos pg ON pg.CODIGO_PEDIDO = p.CODIGO
WHERE pg.CODIGO_PEDIDO IS NULL;








count(1)
3330
